In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [8]:
from creme import metrics
from creme import model_selection
from creme import linear_model
from creme import preprocessing
from creme import datasets

X_y = datasets.Elec2()

model = (
    preprocessing.StandardScaler() |
    linear_model.LogisticRegression()
)

metric = metrics.F1()

for x, y in X_y:
    y_pred = model.predict_one(x)
    metric.update(y, y_pred)
    model.fit_one(x, y)

metric

F1: 0.799183

In [9]:
import collections
from creme.utils import math


class Pegasos:
    
    def __init__(self, lambd):
        self.lambd = lambd
        self.w = collections.defaultdict(float)
    
    def fit_one(self, x, y):
        
        y = y or -1
        
        y_pred = math.dot(x, self.w)
        eta = 0.01
        
        if y * y_pred < 1:
            for i, xi in x.items():
                self.w[i] -= (1 - eta * self.lambd) * self.w[i] + eta * y * xi
        else:
            for i, xi in x.items():
                self.w[i] -= (1 - eta * self.lambd)
                
    def predict_one(self, x):
        return math.dot(x, self.w) > 0

X_y = datasets.Elec2()

model = (
    preprocessing.StandardScaler() |
    Pegasos(0.01)
)

metric = metrics.F1()

for x, y in X_y:
    y_pred = model.predict_one(x)
    metric.update(y, y_pred)
    model.fit_one(x, y)

metric    

F1: 0.147463